In [1]:
from collections import namedtuple
import os
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score,accuracy_score
os.chdir("../")

In [2]:
pwd

'd:\\projects\\credit_default_ml_project'

In [3]:
ModelTrainingConfig = namedtuple("ModelTrainingConfig", [
    "root_dir", 
    "base_accuracy"   
    "model_df_dir",
    "model_df_name"
])

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    base_accuracy: float
    model_df_dir: Path
    model_df_name: str

In [10]:
from Credit_Default.constants import *
from Credit_Default.utils import *

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)      
        
       

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        
        create_directories([config.root_dir,
                            config.model_df_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir), 
            base_accuracy = float(config.base_accuracy),
            model_df_dir = Path(config.model_df_dir),
            model_df_name = str(config.model_df_name)  
        )

        return model_training_config

In [16]:
import os

from pathlib import Path
import pandas as pd
import numpy as np
from Credit_Default.entity import *

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        self.train_array = np.load(TRAIN_ARRAY_FILE_PATH)
        self.test_array = np.load(TEST_ARRAY_FILE_PATH)
        self.model_name=[]
        self.accuracy_score=[]
        self.auc_score=[]

    def logistic_regression_model(self):
        try:
            model=LogisticRegression()
            self.model_name.append('LogisticRegression')
            score=model_score(self.train_array,self.test_array,model)
            self.accuracy_score.append(score[0])
            self.auc_score.append(score[1])
        except Exception as e:
            raise e

    def knn_classifier_model(self):
        try:
            model=KNeighborsClassifier(n_neighbors=3)
            self.model_name.append('KNN_Classifier')
            score=model_score(self.train_array,self.test_array,model)
            self.accuracy_score.append(score[0])
            self.auc_score.append(score[1])
        except Exception as e:
            raise e
    

    def random_forest_classifier_model(self):
        try:
            model=RandomForestClassifier()
            self.model_name.append('Random_Forest_Classifier')
            score=model_score(self.train_array,self.test_array,model)
            self.accuracy_score.append(score[0])
            self.auc_score.append(score[1])
        except Exception as e:
            raise e
    
    def model_dataframe(self):
        try:
            models = pd.DataFrame({
                'Model' : self.model_name,    
                'Accucracy_score' : self.accuracy_score,
                'ROC_AUC_Score' : self.auc_score})
            models['Above_Base_Accuracy']= [True if  models['Accucracy_score'][i] >= self.config.base_accuracy  else False for i in range(len(models))]
            
            model_df_filepath = os.path.join(self.config.model_df_dir,self.config.model_df_name)

            models.to_csv(model_df_filepath,index=False)
        
        except Exception as e:
            raise e

In [17]:
try:
    config = ConfigurationManager()
    model_training_config= config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config) 
    model_training.logistic_regression_model()
    model_training.knn_classifier_model()
    model_training.random_forest_classifier_model()
    model_training.model_dataframe()
 


except Exception as e:
    raise e

c:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\HP\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warn